# Data Preparation

folder_name_image_name_id.jpg
csv_columns = id,image_name,image_width, image_height, object_class_id,object_class_name,bbox_left, bbox_top, bbox_right, bbox_bottom


### operations to be done
- rename the file
- add the data to csv file
- move it to data folder

Directory -- Handler

In [37]:
import pandas as pd
import os

project_directory =  os.getcwd()
data_folder = project_directory + os.sep + 'data'
isExist = os.path.exists(data_folder)
if not isExist:
   os.makedirs(data_folder)
csv_file_path = project_directory + os.sep + "data/image_info.csv"
un_processed_data_path =  project_directory + os.sep + "data-unprocessed"
un_processed_directories = os.listdir(un_processed_data_path)

destination_directory = project_directory + os.sep + "data"



In [38]:
header = {
    "id":[],
    "image_name":[],
    "image_width":[],
    "image_height":[],
    "object_class_id":[],
    "object_class_name":[],
    "bbox_left":[],
    "bbox_top":[],
    "bbox_right":[],
    "bbox_bottom":[],
}


df = pd.DataFrame(header)
# df.to_csv(csv_file_path)
# df.head()

In [39]:
# Add data to csv file

# data_append = {
#     "id":[1],
#     "image_name":["test.jpg"],
#     "image_width":[1989],
#     "image_height":[1211],
#     "object_class_id":[1001],
#     "object_class_name":["byclce"],
#     "bbox_left":[12],
#     "bbox_top":[12],
#     "bbox_right":[21],
#     "bbox_bottom":[21],
# }

# data_append2 = {
#     "id":[2],
#     "image_name":["teassst.jpg"],
#     "image_width":[1989],
#     "image_height":[1211],
#     "object_class_id":[1001],
#     "object_class_name":["byclce"],
#     "bbox_left":[12],
#     "bbox_top":[12],
#     "bbox_right":[21],
#     "bbox_bottom":[21],
# }

# df2 = pd.DataFrame(data_append)
# df4 = pd.DataFrame(data_append2)
# df3 = pd.concat([df, df2, df4], ignore_index=True)
# df3



In [40]:
def connected_data_generator(un_processed_directories):
    unp_data = []

    for dir in un_processed_directories:

        labels =  os.listdir(un_processed_data_path + os.sep + dir + os.sep + "labels" + os.sep + "json")
        images =  os.listdir(un_processed_data_path + os.sep + dir + os.sep + "images")


        coupled_data = list((zip(images,labels)))

        mapped_data = {"dir":dir, "couples":coupled_data}

        unp_data.append(mapped_data)
    
    return unp_data

connected_data = connected_data_generator(un_processed_directories)


In [41]:
from PIL import Image
import shutil
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes

def generate_data_frame(connected_data, un_processed_data_path, df):

    data_append = {            
                "image_name":[],
                "image_width":[],
                "image_height":[],
                "object_class_id":[],
                "object_class_name":[],
                "bbox_left":[],
                "bbox_top":[],
                "bbox_right":[],
                "bbox_bottom":[],
            }

    for dataindex, data in enumerate(connected_data):
        directory = data["dir"]
        couples = data['couples']

        max_limit = 20

        
        for idx, coupled_data in enumerate(couples):
            if idx < max_limit:
                image_file_name = coupled_data[0]
                image_label_json = coupled_data[1]
    
                image_file_path = os.path.join(un_processed_data_path, directory,"images", image_file_name)
                image_label_path = os.path.join(un_processed_data_path, directory,"labels", "json", image_label_json)

                # read the json data
                json_data_frame = pd.read_json(image_label_path)
                object_class_name = json_data_frame['ObjectClassName'].values[0]
                object_class_id = json_data_frame['ObjectClassId'].values[0]
                bbox_left = json_data_frame['Left'].values[0]
                bbox_top  = json_data_frame['Top'].values[0]
                bbox_right  = json_data_frame['Right'].values[0]
                bbox_bottom  = json_data_frame['Bottom'].values[0]
    
                img = Image.open(image_file_path)
        
                width = img.width
                height = img.height

                # data_append['id'].append(idx)
                data_append['image_name'].append(directory+"__"+image_file_name)
                data_append['image_width'].append(width)
                data_append['image_height'].append(height)
                data_append['object_class_id'].append(object_class_id)
                data_append['object_class_name'].append(object_class_name)
                data_append['bbox_left'].append(bbox_left)
                data_append['bbox_top'].append(bbox_top)
                data_append['bbox_right'].append(bbox_right)
                data_append['bbox_bottom'].append(bbox_bottom)
                
                shutil.copy(image_file_path, os.path.join(destination_directory,directory+"__"+image_file_name ))   
            else:
                break    
            
    
    df2  = pd.DataFrame(data_append)
    return df2

def draw_bouding_box(image_file_path, bbox_left, bbox_top, bbox_right, bbox_bottom):
    img = read_image(image_file_path)

    bbox = [bbox_left, bbox_top, bbox_right, bbox_bottom]
    bbox = torch.tensor(bbox, dtype=torch.int)
    print(bbox)
    print(bbox.size())
    bbox = bbox.unsqueeze(0)
    print(bbox.size())

            # draw bounding box on the input image
    img=draw_bounding_boxes(img, bbox, width=3, colors=(255,255,0))

            # transform it to PIL image and display
    img = torchvision.transforms.ToPILImage()(img)
    img.show()
    return img


dfc= generate_data_frame(connected_data, un_processed_data_path, df )
dfc.head()
dfc.to_csv(csv_file_path)